In [6]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import lxml.html as lh
import re



In [7]:
url = 'https://wallethub.com/edu/happiest-states/6959'

# Scrape

In [8]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')
print(soup.prettify())

<!-- PC [U] 2022-04-05T02:10:17.900Z -->
<!DOCTYPE html>
<html lang="en" scroll-region="active">
 <head>
  <link href="https://cdn.wallethub.com" rel="preconnect"/>
  <link href="https://www.googletagmanager.com" rel="preconnect"/>
  <link href="https://www.google-analytics.com" rel="preconnect"/>
  <link href="https://hello.myfonts.net" rel="preconnect"/>
  <link href="https://cdn.wallethub.com" rel="dns-prefetch"/>
  <link href="https://www.googletagmanager.com" rel="dns-prefetch"/>
  <link href="https://www.google-analytics.com" rel="dns-prefetch"/>
  <link href="https://hello.myfonts.net" rel="dns-prefetch"/>
  <link href="https://d2e70e9yced57e.cloudfront.net/common/wallethub/favicon2.ico" rel="shortcut icon"/>
  <link as="font" crossorigin="anonymous" href="https://cdn.wallethub.com/css/wh2015/framework/webfonts/proxima-nova/2D9E95_2_0_V60fad39_.woff2" rel="preload" type="font/woff2"/>
  <link as="image" href="https://cdn.wallethub.com/wallethub/posts/98271/happiest-states-in-ame

In [9]:
results= soup.find('div', class_='cardhub-edu-table-div')
print(results)

<div class="cardhub-edu-table-div sortable-main-1"><table class="cardhub-edu-table center-aligned sortable" style="width: 680px">
<thead>
<tr>
<th class="rank-numeric"><b>Overall Rank*</b></th>
<th><b>State</b></th>
<th class="rank-numeric"><b>Total Score</b></th>
<th class="rank-numeric"><b>Emotional &amp; Physical Well-Being</b></th>
<th class="rank-numeric"><b>Work Environment</b></th>
<th class="rank-numeric"><b>Community &amp; Environment</b></th>
</tr>
</thead>
<tbody><tr><td>1</td><td>Utah</td><td>72.94</td><td>9</td><td>1</td><td>1</td></tr><tr><td>2</td><td>Minnesota</td><td>67.52</td><td>2</td><td>4</td><td>9</td></tr><tr><td>3</td><td>Hawaii</td><td>66.16</td><td>8</td><td>15</td><td>3</td></tr><tr><td>4</td><td>California</td><td>63.82</td><td>4</td><td>20</td><td>11</td></tr><tr><td>5</td><td>North Dakota</td><td>63.25</td><td>13</td><td>5</td><td>6</td></tr><tr><td>6</td><td>South Dakota</td><td>63.08</td><td>14</td><td>6</td><td>4</td></tr><tr><td>7</td><td>Idaho</td><td

In [10]:
# the head will form our column names
body = results.find_all("tr")
# Head values (Column names) are the first items of the body list
head = body[0] # 0th item is the header row
body_rows = body[1:] # All other items becomes the rest of the rows

# Lets now iterate through the head HTML code and make list of clean headings

# Declare empty list to keep Columns names
headings = []
for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    # append the clean column name to headings
    headings.append(item)
print(headings)

['Overall Rank*', 'State', 'Total Score', 'Emotional & Physical Well-Being', 'Work Environment', 'Community & Environment']


In [11]:
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

In [12]:
df = pd.DataFrame(data=all_rows,columns=headings)
df.head()

Overall Rank*         State Total Score Emotional & Physical Well-Being  \
0             1          Utah       72.94                               9   
1             2     Minnesota       67.52                               2   
2             3        Hawaii       66.16                               8   
3             4    California       63.82                               4   
4             5  North Dakota       63.25                              13   

  Work Environment Community & Environment  
0                1                       1  
1                4                       9  
2               15                       3  
3               20                      11  
4                5                       6

In [13]:
df.to_csv("happiness.csv", index= False)

In [14]:
# Add a placeholder for DC with nan values
df.loc[len(df.index)] = ['51', 'District of Columbia', 'NaN', 'NaN','NaN', 'NaN' ] 

In [15]:
df=df.sort_values('State')
df.head()

Overall Rank*       State Total Score Emotional & Physical Well-Being  \
42            43     Alabama       42.99                              45   
41            42      Alaska       44.24                              36   
26            27     Arizona       52.76                              27   
48            49    Arkansas       37.47                              50   
3              4  California       63.82                               4   

   Work Environment Community & Environment  
42               33                      16  
41               41                      48  
26               28                      33  
48               35                      15  
3                20                      11

In [16]:
df.reset_index(inplace=True)

In [21]:
# df= df.drop(columns="index")
df.head()

Overall Rank*       State Total Score Emotional & Physical Well-Being  \
0            43     Alabama       42.99                              45   
1            42      Alaska       44.24                              36   
2            27     Arizona       52.76                              27   
3            49    Arkansas       37.47                              50   
4             4  California       63.82                               4   

  Work Environment Community & Environment  
0               33                      16  
1               41                      48  
2               28                      33  
3               35                      15  
4               20                      11

In [25]:
df=df.rename(columns={"Overall Rank*": "Happiness Rank", "Total Score": "Happiness Score", "Emotional & Physical Well-Being": "Emotional & Physical Well-Being Rank", "Work Environment": "Work Environment Rank", "Community & Environment": "Community & Environment Rank"})

In [26]:
df.to_json('happiness.json', orient='records')